In [3]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
import os
import json

In [4]:
parks_geoparquet = "data" + os.sep + "parks.geoparquet"
zones_geoparquet = "data" + os.sep + "zones.geoparquet"

In [5]:
parks = gpd.read_parquet(parks_geoparquet)
zones = gpd.read_parquet(zones_geoparquet)

In [6]:
zones.explore()

In [ ]:
# ['id', 'driver', 'geom', 'name', 'type', 'city', 'color', 'capacity',
#       'freeslots', 'diag', 'threshold', 'timestamp', 'currentTimestamp',
#       'address', 'gmaps', 'link', 'offline', 'opening', 'open', 'phone',
#       'website', 'openingFlag', 'opening_lunven', 'opening_sab',
#       'opening_fes'],
#      dtype='object')

In [ ]:
total_carparkspaces = parks[parks['type'] == 'park'].capacity.sum()
total_carparkspaces_free = parks[parks['type'] == 'park'].freeslots.sum()

In [ ]:
carparks[carparks.currentTimestamp == carparks.currentTimestamp.max()].capacity.sum()

2350

In [ ]:
total_bikeparkspaces = parks[parks['type'] == 'bike'].capacity.sum()
total_bikeparkspaces_free = parks[parks['type'] == 'bike'].freeslots.sum()

In [ ]:
parks.timestamp.max()

Timestamp('2025-03-12 22:18:43')

In [ ]:
parks.currentTimestamp.max()

Timestamp('2025-03-12 22:19:01')

In [ ]:
total_carparkspaces_free

76299

In [2]:
zones.explore()

NameError: name 'zones' is not defined

In [ ]:
round(total_carparkspaces_free/total_carparkspaces,2)

0.33

In [ ]:
# Raggruppare dati ogni 15 o 30 minuti (modificare freq se necessario)
parks_time_grouped = parks.groupby([
    pd.Grouper(key='timestamp', freq='15min'), 'name'
]).agg({'freeslots': 'mean', 'capacity': 'first'}).reset_index()

In [ ]:
parks_time_grouped

,timestamp,name,freeslots,capacity
0,2025-03-11 10:00:00,Garage Autosilo Buonconsiglio - P3,0.0,188
1,2025-03-11 10:00:00,Garage Centro Europa - P2,11.0,330
2,2025-03-11 10:00:00,Garage piazza Fiera - P4,46.0,448
3,2025-03-11 10:00:00,Parcheggio Cittadella dello studente - P6,116.0,145
4,2025-03-11 10:00:00,Parcheggio piazzale Sanseverino - P7,0.0,397
...,...,...,...,...
1461,2025-03-12 22:15:00,Parcheggio via Monte Baldo - P8,186.0,202
1462,2025-03-12 22:15:00,Piedicastello,0.0,26
1463,2025-03-12 22:15:00,Stazione di Trento 2,26.0,70
1464,2025-03-12 22:15:00,Via Bartali,10.0,12


In [ ]:
# Preparazione dei dati per ECharts
data_echarts = {
    'timestamps': sorted(parks_time_grouped['timestamp'].dt.strftime('%Y-%m-%d %H:%M').unique().tolist()),
    'series': []
}

for name in parks_time_grouped['name'].unique():
    subset = parks_time_grouped[parks_time_grouped['name'] == name].sort_values(by='timestamp')
    data_echarts['series'].append({
        'name': name,
        'type': 'line',
        'data': subset['freeslots'].tolist()
    })
    data_echarts['series'].append({
        'name': f"{name} (Capacità)",
        'type': 'line',
        'data': subset['capacity'].tolist(),
        'lineStyle': {'type': 'dashed'}
    })

# Esportare i dati in un file JSON
with open('docs/data/parking_data.json', 'w') as f:
    json.dump(data_echarts, f, indent=2)

